# AI Powered Situation Puzzle Game #

Save your OpenAI api key to a file named `openai_api_key.txt`
<br>
You can add more games by storing new "scenario" and "truth" pairs into `games.json`

In [14]:
%pip install openai


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [15]:
import openai
import json

openai.api_key_path = "openai_api_key.txt"

In [16]:
f = open('games.json')
games = json.load(f)

def get_num_games():
    return len(games['games'])

def get_scenario(index):
    return games['games'][index]['scenario']

def get_truth(index):
    return games['games'][index]['truth']

In [17]:
rule_template = """
You are the game master of a situation puzzle game.
You will receive a pair of paragraphs, which constitue a complete story.
The player will ask a series of dichotomous questions about the story.
Your role is to respond to the player's questions by referencing the paragraphs you are given, mainly the second paragraph.

Your goal is to respond in a manner that aligns with either "yes" or "no." Your answer should convey a clear affirmative or negative meaning.
If the answer cannot be determined from the provided information, respond in a way that reflects this uncertainty, like "irrelevant".
If the question is not a dichotomous question, respond "This cannot be answered with simply yes or no", and prompt the player to rephrase the question.
If the question is blank, which means the player does not enter a question, respond "Enter a question".

Your responses should strictly adhere to these rules and options.
"""

puzzle_template = """
{scenario}

{truth}
"""

restoration_rule_template = """
You are the game master of a situation puzzle game.
Your task is to compare the player's proposed truth with the actual truth provided.
Follow these guidelines to assess the player's response effectively:

1. Analyze the actual truth and identify its key points. Summarize these key points concisely.

2. Analyze the player's proposed truth. Summarize the key points of the player's proposed truth.

3. Compare the summarized key points of the player's proposed truth with the key points of the actual truth. Assess if the player's proposal matches all key points or if any are missing.

Use these assessments to guide your response:
- If the player's proposed truth covers all key points of the actual truth, say that they have succeed.
- If the player's proposed truth is missing key points, say that their answer is not emtirely correct.

Your role is crucial in determining if the player's proposed truth aligns with the actual truth. Follow these instructions diligently for fair and accurate assessments.
"""

restoration_template = """
Player's proposed truth: {proposed_truth}

Actual truth: {truth}
"""


In [18]:
def run_games():
    num_games = get_num_games()
    print(f"There are {num_games} games.")

    for i in range(num_games):
        scenario = get_scenario(i)
        truth = get_truth(i)
        print("-----------------------------------------------")
        print(f"This is game {i + 1}")
        print(f"The scenario is: {scenario}")

        conversation_history = []
        conversation_history.append({"role": "system", "content": rule_template})
        conversation_history.append({"role": "system", "content": puzzle_template.format(scenario=scenario, truth=truth)})
        questioning(conversation_history, truth)

# the system does not record the questions and answers
def questioning(conversation_history, truth):
    print("Now, you can start questioning. Type quit to stop questioning and move on to restore the truth.")
    while True:
        user_input = input("Your question: ")
        if user_input.lower() == "quit":
            break
        print(f"You: {user_input}")
        
        conversation_history.append({"role": "user", "content": user_input})
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=conversation_history,
            temperature=0
        )
        system_reply = response.choices[0].message["content"]
        print(f"Game Master: {system_reply}")
        conversation_history.pop() # pop the newly appended user_input, which deletes the question from the history

    restoring(truth)

def restoring(truth):
    print("-----------------------------------------------")
    print("Now, you can start restoring the truth.")
    user_input = input("What happened: ")
    print(f"What you think happened: {user_input}")

    conversation_history = []
    conversation_history.append({"role": "system", "content": restoration_rule_template})
    conversation_history.append({"role": "user", "content": restoration_template.format(proposed_truth=user_input, truth=truth)})
    response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=conversation_history,
            temperature=0
        )
        
    system_reply = response.choices[0].message["content"]
    print(f"Game Master: {system_reply}")
    print(f"The truth: {truth}")


## Run the game by running the next block. ##

In [19]:
run_games()

There are 2 games.
-----------------------------------------------
This is game 1
The scenario is: A man walks into a bar and asks the bartender for a glass of water. The bartender pulls out a gun and points it at the man. The man says, "Thank you" and walks out.
Now, you can start questioning. Type quit to stop questioning and move on to restore the truth.
You: does the bartender want to kill the man
Game Master: No.
You: 
Game Master: Enter a question.
You: is the man sick
Game Master: Irrelevant.
You: does the man have hiccups
Game Master: Yes.
-----------------------------------------------
Now, you can start restoring the truth.
What you think happened: The man has hiccups. The bartender uses the gun to scare the hiccups out of the man.
Game Master: Key points of the actual truth:
1. The man had hiccups.
2. The gun scared the hiccups out of him.
3. The man expressed gratitude by saying, "Thank you."

Key points of the player's proposed truth:
1. The man had hiccups.
2. The bartend